In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import quantecon as qe
import matplotlib.pyplot as plt
import pickle

***Load Datasets***

In [2]:
df = pd.read_stata("/Users/jakesharadin/Desktop/berkeley/2024_3_fall/econ_191/research/code/final_data.dta")
df.head()

,county,schedule1_count,schedule2_count,total_pop,log_slave,ag_output,ag_gdp,gini,cash_value_farms,state,county_name
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,Autauga
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,Baldwin
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,Barbour
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,Bibb
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,Blount


In [3]:
df_2010_sorted = df.sort_values(by='schedule1_count').reset_index(drop=True)
df_2010_sorted

,county,schedule1_count,schedule2_count,total_pop,log_slave,ag_output,ag_gdp,gini,cash_value_farms,state,county_name
0,3050.0,1648.0,7.0,1655.0,1.945910,69372.0,41.916615,0.00000,145633.0,Georgia,Wayne
1,3150.0,1694.0,92.0,1786.0,4.521789,392609.0,219.825867,0.05705,645612.0,Georgia,Wilcox
2,2650.0,1733.0,195.0,1928.0,5.273000,312827.0,162.254669,0.38865,744580.0,Georgia,Taliaferro
3,2990.0,1823.0,21.0,1844.0,3.044523,85240.0,46.225597,0.30360,474829.0,Georgia,Ware
4,2710.0,1876.0,44.0,1920.0,3.784190,184218.0,95.946877,0.48880,299145.0,Georgia,Telfair
...,...,...,...,...,...,...,...,...,...,...,...
147,250.0,305082.0,1612.0,306694.0,7.385231,25603595.0,83.482544,0.55802,98906678.0,Alabama,Clarke
148,170.0,352617.0,2489.0,355106.0,7.819636,35313019.0,99.443604,0.57543,161524985.0,Alabama,Chambers
149,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,Autauga
150,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,Blount


In [4]:
#152 observations (counties)
df.shape

(152, 11)

In [5]:
# Removed rows were "cash_value_farms" is zero. It's okay to remove them because, aftering further inspection of the rows 
# we can see counties with farm cash value of 0, also have population totals of 0. except county #5100. This county has 
# a population of 60,000+ so we assume this must be an error and remove all counties with farm cash value of 0.

# Replaced missing schedule1 and schedule2 population values (NaN) with 0. Some counties have slave populations but no freeman. Others have 
# freeman but no slaves. I believe this is the best solution

# Removed rows in which the total population is zero but the cash value of farms is non-zero as these datapoints could outwardly
# influnce the gini coefficent.

In [6]:
# Add log population variable
df["log_pop"] = np.log(df["total_pop"].replace(0, np.nan))
df.head()

,county,schedule1_count,schedule2_count,total_pop,log_slave,ag_output,ag_gdp,gini,cash_value_farms,state,county_name,log_pop
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,Autauga,12.804104
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,Baldwin,12.590995
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,Barbour,12.891712
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,Bibb,11.840364
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,Blount,12.837826


In [7]:
gini_county = pd.read_stata("/Users/jakesharadin/Desktop/berkeley/2024_3_fall/econ_191/research/data/gini_(inequality)/1860/gini_county.dta")
df_v1 = pd.merge(df, gini_county, on='county', how='inner')
df_v1.rename(columns={"gini_y": "gini_stata"}, inplace=True)
df_v1

,county,schedule1_count,schedule2_count,total_pop,log_slave,ag_output,ag_gdp,gini_x,cash_value_farms,state,county_name,log_pop,gini_stata
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,Autauga,12.804104,0.54031
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,Baldwin,12.590995,0.58392
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,Barbour,12.891712,0.52155
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,Bibb,11.840364,0.43734
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,Blount,12.837826,0.59723
...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,3130.0,8315.0,170.0,8485.0,5.135798,471886.0,55.614143,0.35042,1818620.0,Georgia,Whitfield,9.046055,0.35042
148,3150.0,1694.0,92.0,1786.0,4.521789,392609.0,219.825867,0.05705,645612.0,Georgia,Wilcox,7.487734,0.05705
149,3170.0,3466.0,745.0,4211.0,6.613384,770411.0,182.952026,0.00000,1601158.0,Georgia,Wilkes,8.345455,0.00000
150,3190.0,5499.0,346.0,5845.0,5.846439,950803.0,162.669464,0.48048,2013324.0,Georgia,Wilkinson,8.673342,0.48048


In [8]:
# Drop all gini coefficients with value zero
df_v1 = df_v1[df_v1['gini_stata'] != 0]
df_v1.shape

(137, 13)

In [9]:
# Final check for duplicates, appears to be none.
duplicates = df_v1[df_v1.duplicated(subset=['county_name', 'state'], keep=False)]
duplicates

,county,schedule1_count,schedule2_count,total_pop,log_slave,ag_output,ag_gdp,gini_x,cash_value_farms,state,county_name,log_pop,gini_stata


In [10]:
# State abbreviation column
# Define a dictionary to map state names to their abbreviations
state_to_abbreviation = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL',
    'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI',
    'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND',
    'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
    'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Create a new column "state_abb" in df_v1 using the mapping dictionary
df_v1['state_abb'] = df_v1['state'].map(state_to_abbreviation)

# County_state for crime data merging
df_v1['county_state'] = df_v1['county_name'] + ', ' + df_v1['state_abb']
df_v1.head()

/var/folders/vl/4mchf1rx7l7dljxrjt1ct0_80000gn/T/ipykernel_3012/3848363871.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_v1['state_abb'] = df_v1['state'].map(state_to_abbreviation)
/var/folders/vl/4mchf1rx7l7dljxrjt1ct0_80000gn/T/ipykernel_3012/3848363871.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_v1['county_state'] = df_v1['county_name'] + ', ' + df_v1['state_abb']


,county,schedule1_count,schedule2_count,total_pop,log_slave,ag_output,ag_gdp,gini_x,cash_value_farms,state,county_name,log_pop,gini_stata,state_abb,county_state
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,Autauga,12.804104,0.54031,AL,"Autauga, AL"
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,Baldwin,12.590995,0.58392,AL,"Baldwin, AL"
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,Barbour,12.891712,0.52155,AL,"Barbour, AL"
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,Bibb,11.840364,0.43734,AL,"Bibb, AL"
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,Blount,12.837826,0.59723,AL,"Blount, AL"


In [11]:
# Manual calculation of gini coefficent - come back to this

In [12]:
# Save dataset before entering crime data
with open('df_v1.pkl', 'wb') as f:
    pickle.dump(df_v1, f)

*** Crime Data ***

In [13]:
# 2016 crime data, 2013 population data, #3,136 counties
crime_data = pd.read_csv("/Users/jakesharadin/Desktop/berkeley/2024_3_fall/econ_191/research/data/crime/crime_data_w_population_and_crime_rate.csv")
crime_data.head()

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,200,1778,3609,4995,13791,3543,464,318416,29,510
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,38,165,662,1482,1753,189,28,49746,5,35
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,2,5,119,82,184,12,2,7629,17,3
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,3,1,2,5,4,4,0,412,48,261
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,4,17,368,149,494,60,0,27083,22,31


In [14]:
# Split 'county_name' into the main county name and the state abbreviation
crime_data[['county_name_clean', 'state_abb']] = crime_data['county_name'].str.rsplit(', ', n=1, expand=True)

# Remove "County" if it's the second word and capitalize each word in 'county_name_clean'
crime_data['county_name_clean'] = (
    crime_data['county_name_clean']
    .str.replace(r'\b County\b', '', regex=True)  # Remove "County" if it appears as the second word
    .str.title()                                  # Capitalize each word
)

# Combine the cleaned 'county_name_clean' with 'state_abb' to create 'county_state'
crime_data['county_state'] = crime_data['county_name_clean'] + ', ' + crime_data['state_abb']

# Drop the temporary 'county_name_clean' and 'state_abb' columns if no longer needed
crime_data = crime_data.drop(columns=['county_name_clean', 'state_abb'])

# Display the first few rows to verify the changes
crime_data.head()


,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY,county_state
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,1778,3609,4995,13791,3543,464,318416,29,510,"St. Louis City, MO"
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,165,662,1482,1753,189,28,49746,5,35,"Crittenden, AR"
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,5,119,82,184,12,2,7629,17,3,"Alexander, IL"
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,1,2,5,4,4,0,412,48,261,"Kenedy, TX"
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,17,368,149,494,60,0,27083,22,31,"De Soto Parish, LA"


In [15]:
# Merge 2016 crime and 1860 census data
merged_df = df_v1.merge(crime_data, on="county_state", how="inner")
merged_df.shape
#merged_df.head()

(133, 39)

In [16]:
merged_df['sum_crime'] = (
    merged_df['MURDER'] + 
    merged_df['RAPE'] + 
    merged_df['ROBBERY'] + 
    merged_df['AGASSLT'] + 
    merged_df['BURGLRY'] + 
    merged_df['LARCENY'] + 
    merged_df['MVTHEFT'] + 
    merged_df['ARSON']
)

merged_df.head()

,county,schedule1_count,schedule2_count,total_pop,log_slave,ag_output,ag_gdp,gini_x,cash_value_farms,state,...,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY,sum_crime
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,...,34,87,447,1233,85,108,55246,1,1,2012
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,...,76,332,967,3829,192,31,195540,1,3,5465
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,...,8,36,90,362,21,0,27076,1,5,521
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,...,8,36,122,251,27,0,22512,1,7,449
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,...,9,101,397,865,86,9,57872,1,9,1479


In [17]:
merged_df['log_sum_crime'] = np.log(merged_df['sum_crime'] + 1)
merged_df.head()

,county,schedule1_count,schedule2_count,total_pop,log_slave,ag_output,ag_gdp,gini_x,cash_value_farms,state,...,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY,sum_crime,log_sum_crime
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,...,87,447,1233,85,108,55246,1,1,2012,7.607381
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,...,332,967,3829,192,31,195540,1,3,5465,8.606302
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,...,36,90,362,21,0,27076,1,5,521,6.257668
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,...,36,122,251,27,0,22512,1,7,449,6.109248
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,...,101,397,865,86,9,57872,1,9,1479,7.299797


In [18]:
# Save merged_df_clean to a pickle file
with open('merged_df.pkl', 'wb') as f:
    pickle.dump(merged_df, f)

***2010 Crime Data***

In [19]:
# Merge 2010 population with 2010 crime data
crime_2010 = pd.read_stata("/Users/jakesharadin/Desktop/berkeley/2024_3_fall/econ_191/research/data/crime/2010/2010_crime.dta")
crime_2010.head()

,FIPS_ST,FIPS_CTY,GRNDTOT,MURDER,RAPE,MVTHEFT
0,1,1,2710,5,1,10
1,1,3,8503,1,6,30
2,1,5,707,0,0,0
3,1,7,780,1,2,8
4,1,9,1979,2,2,8


In [20]:
# Define state abbreviations
state_fips_abbrev = {
    1: "AL",   # Alabama
    5: "AR",   # Arkansas
    13: "GA",  # Georgia
    17: "IL",  # Illinois
    21: "KY",  # Kentucky
    6: "CA"    # California
}

# Define county names based on FIPS codes
county_fips_names = {
    1: {  # Alabama
        1: "Autauga", 3: "Baldwin", 5: "Barbour", 7: "Bibb", 9: "Blount", 11: "Bullock", 13: "Butler",
        15: "Calhoun", 17: "Chambers", 19: "Cherokee", 21: "Chilton", 23: "Choctaw", 25: "Clarke",
        27: "Clay", 29: "Cleburne", 31: "Coffee", 33: "Colbert", 35: "Conecuh", 37: "Coosa",
        39: "Covington", 41: "Crenshaw", 43: "Cullman", 45: "Dale", 47: "Dallas", 49: "DeKalb",
        51: "Elmore", 53: "Escambia", 55: "Etowah", 57: "Fayette", 59: "Franklin", 61: "Geneva",
        63: "Greene", 65: "Hale", 67: "Henry", 69: "Houston", 71: "Jackson", 73: "Jefferson",
        75: "Lamar", 77: "Lauderdale", 79: "Lawrence", 81: "Lee", 83: "Limestone", 85: "Lowndes",
        87: "Macon", 89: "Madison", 91: "Marengo", 93: "Marion", 95: "Marshall", 97: "Mobile",
        99: "Monroe", 101: "Montgomery", 103: "Morgan", 105: "Perry", 107: "Pickens", 109: "Pike",
        111: "Randolph", 113: "Russell", 115: "St. Clair", 117: "Shelby", 119: "Sumter",
        121: "Talladega", 123: "Tallapoosa", 125: "Tuscaloosa", 127: "Walker", 129: "Washington",
        131: "Wilcox", 133: "Winston"
    },
    5: {  # Arkansas
        1: "Arkansas County", 3: "Ashley", 5: "Baxter", 7: "Benton", 9: "Boone", 11: "Bradley",
        13: "Calhoun", 15: "Carroll", 17: "Chicot", 19: "Clark", 21: "Clay", 23: "Cleburne",
        25: "Cleveland", 27: "Columbia", 29: "Conway", 31: "Craighead", 33: "Crawford", 35: "Crittenden",
        37: "Cross", 39: "Dallas", 41: "Desha", 43: "Drew", 45: "Faulkner", 47: "Franklin",
        49: "Fulton", 51: "Garland", 53: "Grant", 55: "Greene", 57: "Hempstead", 59: "Hot Spring",
        61: "Howard", 63: "Independence", 65: "Izard", 67: "Jackson", 69: "Jefferson", 71: "Johnson",
        73: "Lafayette", 75: "Lawrence", 77: "Lee", 79: "Lincoln", 81: "Little River", 83: "Logan",
        85: "Lonoke", 87: "Madison", 89: "Marion", 91: "Miller", 93: "Mississippi", 95: "Monroe",
        97: "Montgomery", 99: "Nevada", 101: "Newton", 103: "Ouachita", 105: "Perry", 107: "Phillips",
        109: "Pike", 111: "Poinsett", 113: "Polk", 115: "Pope", 117: "Prairie", 119: "Pulaski",
        121: "Randolph", 123: "St. Francis", 125: "Saline", 127: "Scott", 129: "Searcy",
        131: "Sebastian", 133: "Sevier", 135: "Sharp", 137: "Stone", 139: "Union", 141: "Van Buren",
        143: "Washington", 145: "White", 147: "Woodruff", 149: "Yell"
    },

    13: {  # Georgia
        1: "Appling", 3: "Atkinson", 5: "Bacon", 7: "Baker", 9: "Baldwin", 11: "Banks",
        13: "Barrow", 15: "Bartow", 17: "Ben Hill", 19: "Berrien", 21: "Bibb", 23: "Bleckley",
        25: "Brantley", 27: "Brooks", 29: "Bryan", 31: "Bulloch", 33: "Burke", 35: "Butts",
        37: "Calhoun", 39: "Camden", 43: "Candler", 45: "Carroll", 47: "Catoosa", 49: "Charlton",
        51: "Chatham", 53: "Chattahoochee", 55: "Chattooga", 57: "Cherokee", 59: "Clarke",
        61: "Clay", 63: "Clayton", 65: "Clinch", 67: "Cobb", 69: "Coffee", 71: "Colquitt",
        73: "Columbia", 75: "Cook", 77: "Coweta", 79: "Crawford", 81: "Crisp", 83: "Dade",
        85: "Dawson", 87: "Decatur", 89: "DeKalb", 91: "Dodge", 93: "Dooly", 95: "Dougherty",
        97: "Douglas", 99: "Early", 101: "Echols", 103: "Effingham", 105: "Elbert", 107: "Emanuel",
        109: "Evans", 111: "Fannin", 113: "Fayette", 115: "Floyd", 117: "Forsyth", 119: "Franklin",
        121: "Fulton", 123: "Gilmer", 125: "Glascock", 127: "Glynn", 129: "Gordon", 131: "Grady",
        133: "Greene", 135: "Gwinnett", 137: "Habersham", 139: "Hall", 141: "Hancock", 143: "Haralson",
        145: "Harris", 147: "Hart", 149: "Heard", 151: "Henry", 153: "Houston", 155: "Irwin",
        157: "Jackson", 159: "Jasper", 161: "Jeff Davis", 163: "Jefferson", 165: "Jenkins", 167: "Johnson",
        169: "Jones", 171: "Lamar", 173: "Lanier", 175: "Laurens", 177: "Lee", 179: "Liberty",
        181: "Lincoln", 183: "Long", 185: "Lowndes", 187: "Lumpkin", 189: "McDuffie",
        191: "McIntosh", 193: "Macon", 195: "Madison", 197: "Marion", 199: "Meriwether",
        201: "Miller", 205: "Mitchell", 207: "Monroe", 209: "Montgomery", 211: "Morgan",
        213: "Murray", 215: "Muscogee", 217: "Newton", 219: "Oconee", 221: "Oglethorpe",
        223: "Paulding", 225: "Peach", 227: "Pickens", 229: "Pierce", 231: "Pike", 233: "Polk",
        235: "Pulaski", 237: "Putnam", 239: "Quitman", 241: "Rabun", 243: "Randolph",
        245: "Richmond", 247: "Rockdale", 249: "Schley", 251: "Screven", 253: "Seminole",
        255: "Spalding", 257: "Stephens", 259: "Stewart", 261: "Sumter", 263: "Talbot",
        265: "Taliaferro", 267: "Tattnall", 269: "Taylor", 271: "Telfair", 273: "Terrell",
        275: "Thomas", 277: "Tift", 279: "Toombs", 281: "Towns", 283: "Treutlen", 285: "Troup",
        287: "Turner", 289: "Twiggs", 291: "Union", 293: "Upson", 295: "Walker", 297: "Walton",
        299: "Ware", 301: "Warren", 303: "Washington", 305: "Wayne", 307: "Webster",
        309: "Wheeler", 311: "White", 313: "Whitfield", 315: "Wilcox", 317: "Wilkes",
        319: "Wilkinson", 321: "Worth"
    },

    17: {  # Illinois
        1: "Adams", 3: "Alexander", 5: "Bond", 7: "Boone", 9: "Brown", 11: "Bureau", 13: "Calhoun",
        15: "Carroll", 17: "Cass", 19: "Champaign", 21: "Christian", 23: "Clark", 25: "Clay",
        27: "Clinton", 29: "Coles", 31: "Cook", 33: "Crawford", 35: "Cumberland", 37: "DeKalb",
        39: "De Witt", 41: "Douglas", 43: "DuPage", 45: "Edgar", 47: "Edwards", 49: "Effingham",
        51: "Fayette", 53: "Ford", 55: "Franklin", 57: "Fulton", 59: "Gallatin", 61: "Greene",
        63: "Grundy", 65: "Hamilton", 67: "Hancock", 69: "Hardin", 71: "Henderson", 73: "Henry",
        75: "Iroquois", 77: "Jackson", 79: "Jasper", 81: "Jefferson", 83: "Jersey", 85: "Jo Daviess",
        87: "Johnson", 89: "Kane", 91: "Kankakee", 93: "Kendall", 95: "Knox", 97: "Lake", 99: "LaSalle",
        101: "Lawrence", 103: "Lee", 105: "Livingston", 107: "Logan", 109: "McDonough", 111: "McHenry",
        113: "McLean", 115: "Macon", 117: "Macoupin", 119: "Madison", 121: "Marion", 123: "Marshall",
        125: "Mason", 127: "Massac", 129: "Menard", 131: "Mercer", 133: "Monroe", 135: "Montgomery",
        137: "Morgan", 139: "Moultrie", 141: "Ogle", 143: "Peoria", 145: "Perry", 147: "Piatt",
        149: "Pike", 151: "Pope", 153: "Pulaski", 155: "Putnam", 157: "Randolph", 159: "Richland",
        161: "Rock Island", 163: "St. Clair", 165: "Saline", 167: "Sangamon", 169: "Schuyler",
        171: "Scott", 173: "Shelby", 175: "Stark", 177: "Stephenson", 179: "Tazewell", 181: "Union",
        183: "Vermilion", 185: "Wabash", 187: "Warren", 189: "Washington", 191: "Wayne", 193: "White",
        195: "Whiteside", 197: "Will", 199: "Williamson", 201: "Winnebago", 203: "Woodford"
    },
    21: {  # Kentucky
        1: "Adair", 3: "Allen", 5: "Anderson", 7: "Ballard", 9: "Barren", 11: "Bath", 13: "Bell",
        15: "Boone", 17: "Bourbon", 19: "Boyd", 21: "Boyle", 23: "Bracken", 25: "Breathitt",
        27: "Breckinridge", 29: "Bullitt", 31: "Butler", 33: "Caldwell", 35: "Calloway", 37: "Campbell",
        39: "Carlisle", 41: "Carroll", 43: "Carter", 45: "Casey", 47: "Christian", 49: "Clark",
        51: "Clay", 53: "Clinton", 55: "Crittenden", 57: "Cumberland", 59: "Daviess", 61: "Edmonson",
        63: "Elliott", 65: "Estill", 67: "Fayette", 69: "Fleming", 71: "Floyd", 73: "Franklin",
        75: "Fulton", 77: "Gallatin", 79: "Garrard", 81: "Grant", 83: "Graves", 85: "Grayson",
        87: "Green", 89: "Greenup", 91: "Hancock", 93: "Hardin", 95: "Harlan", 97: "Harrison",
        99: "Hart", 101: "Henderson", 103: "Henry", 105: "Hickman", 107: "Hopkins", 109: "Jackson",
        111: "Jefferson", 113: "Jessamine", 115: "Johnson", 117: "Kenton", 119: "Knott", 121: "Knox",
        123: "Larue", 125: "Laurel", 127: "Lawrence", 129: "Lee", 131: "Leslie", 133: "Letcher",
        135: "Lewis", 137: "Lincoln", 139: "Livingston", 141: "Logan", 143: "Lyon", 145: "McCracken",
        147: "McCreary", 149: "McLean", 151: "Madison", 153: "Magoffin", 155: "Marion", 157: "Marshall",
        159: "Martin", 161: "Mason", 163: "Meade", 165: "Menifee", 167: "Mercer", 169: "Metcalfe",
        171: "Monroe", 173: "Montgomery", 175: "Morgan", 177: "Muhlenberg", 179: "Nelson", 181: "Nicholas",
        183: "Ohio", 185: "Oldham", 187: "Owen", 189: "Owsley", 191: "Pendleton", 193: "Perry",
        195: "Pike", 197: "Powell", 199: "Pulaski", 201: "Robertson", 203: "Rockcastle", 205: "Rowan",
        207: "Russell", 209: "Scott", 211: "Shelby", 213: "Simpson", 215: "Spencer", 217: "Taylor",
        219: "Todd", 221: "Trigg", 223: "Trimble", 225: "Union", 227: "Warren", 229: "Washington",
        231: "Wayne", 233: "Webster", 235: "Whitley", 237: "Wolfe", 239: "Woodford"
    },

    6: {  # California
        1: "Alameda", 3: "Alpine", 5: "Amador", 7: "Butte", 9: "Calaveras", 11: "Colusa",
        13: "Contra Costa", 15: "Del Norte", 17: "El Dorado", 19: "Fresno", 21: "Glenn",
        23: "Humboldt", 25: "Imperial", 27: "Inyo", 29: "Kern", 31: "Kings", 33: "Lake",
        35: "Lassen", 37: "Los Angeles", 39: "Madera", 41: "Marin", 43: "Mariposa",
        45: "Mendocino", 47: "Merced", 49: "Modoc", 51: "Mono", 53: "Monterey", 55: "Napa",
        57: "Nevada", 59: "Orange", 61: "Placer", 63: "Plumas", 65: "Riverside", 67: "Sacramento",
        69: "San Benito", 71: "San Bernardino", 73: "San Diego", 75: "San Francisco", 77: "San Joaquin",
        79: "San Luis Obispo", 81: "San Mateo", 83: "Santa Barbara", 85: "Santa Clara",
        87: "Santa Cruz", 89: "Shasta", 91: "Sierra", 93: "Siskiyou", 95: "Solano", 97: "Sonoma",
        99: "Stanislaus", 101: "Sutter", 103: "Tehama", 105: "Trinity", 107: "Tulare", 109: "Tuolumne",
        111: "Ventura", 113: "Yolo", 115: "Yuba"
    }
    
}

# Map FIPS_ST to state abbreviation
crime_2010['state_abbrev'] = crime_2010['FIPS_ST'].map(state_fips_abbrev)

# Map FIPS_ST and FIPS_CTY to county name using the county_fips_names dictionary
crime_2010['county_name'] = crime_2010.apply(
    lambda x: county_fips_names.get(x['FIPS_ST'], {}).get(x['FIPS_CTY'], 'Unknown'), axis=1
)

# Display the resulting DataFrame
crime_2010

,FIPS_ST,FIPS_CTY,GRNDTOT,MURDER,RAPE,MVTHEFT,state_abbrev,county_name
0,1,1,2710,5,1,10,AL,Autauga
1,1,3,8503,1,6,30,AL,Baldwin
2,1,5,707,0,0,0,AL,Barbour
3,1,7,780,1,2,8,AL,Bibb
4,1,9,1979,2,2,8,AL,Blount
...,...,...,...,...,...,...,...,...
3172,56,39,755,0,2,3,NaN,Unknown
3173,56,41,1158,2,0,2,NaN,Unknown
3174,56,43,306,0,0,3,NaN,Unknown
3175,56,45,212,0,0,0,NaN,Unknown


In [21]:
# List of FIPS_ST codes for the six states
states_to_include = [1, 5, 13, 17, 21, 6]

# Filter the DataFrame
crime_2010_filtered = crime_2010[crime_2010['FIPS_ST'].isin(states_to_include)]

# Display the filtered DataFrame
crime_2010_filtered #583 counties in 6 states 

,FIPS_ST,FIPS_CTY,GRNDTOT,MURDER,RAPE,MVTHEFT,state_abbrev,county_name
0,1,1,2710,5,1,10,AL,Autauga
1,1,3,8503,1,6,30,AL,Baldwin
2,1,5,707,0,0,0,AL,Barbour
3,1,7,780,1,2,8,AL,Bibb
4,1,9,1979,2,2,8,AL,Blount
...,...,...,...,...,...,...,...,...
1116,21,231,885,0,1,0,KY,Wayne
1117,21,233,564,0,0,0,KY,Webster
1118,21,235,2403,2,6,7,KY,Whitley
1119,21,237,73,0,0,0,KY,Wolfe


In [22]:
# Filter for rows where county_name is "Unknown"
unknown_counties = crime_2010_filtered[crime_2010_filtered['county_name'] == "Unknown"]

# Display the result
unknown_counties.head()

# These seem like mistakes so drop them

,FIPS_ST,FIPS_CTY,GRNDTOT,MURDER,RAPE,MVTHEFT,state_abbrev,county_name
67,1,777,0,0,0,0,AL,Unknown
245,6,777,0,0,0,0,CA,Unknown


In [23]:
# Drop counties with code 777
crime_2010_filtered = crime_2010_filtered[crime_2010['FIPS_CTY'] != 777]

# Display the resulting DataFrame to confirm the rows were removed
crime_2010_filtered.head()

/var/folders/vl/4mchf1rx7l7dljxrjt1ct0_80000gn/T/ipykernel_3012/3232118428.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  crime_2010_filtered = crime_2010_filtered[crime_2010['FIPS_CTY'] != 777]


,FIPS_ST,FIPS_CTY,GRNDTOT,MURDER,RAPE,MVTHEFT,state_abbrev,county_name
0,1,1,2710,5,1,10,AL,Autauga
1,1,3,8503,1,6,30,AL,Baldwin
2,1,5,707,0,0,0,AL,Barbour
3,1,7,780,1,2,8,AL,Bibb
4,1,9,1979,2,2,8,AL,Blount


In [24]:
# Create county_state variable
crime_2010_filtered['county_state'] = crime_2010_filtered['county_name'] + ", " + crime_2010_filtered['state_abbrev']

# Display the DataFrame to confirm the new column
crime_2010_filtered

/var/folders/vl/4mchf1rx7l7dljxrjt1ct0_80000gn/T/ipykernel_3012/2405612588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_2010_filtered['county_state'] = crime_2010_filtered['county_name'] + ", " + crime_2010_filtered['state_abbrev']


,FIPS_ST,FIPS_CTY,GRNDTOT,MURDER,RAPE,MVTHEFT,state_abbrev,county_name,county_state
0,1,1,2710,5,1,10,AL,Autauga,"Autauga, AL"
1,1,3,8503,1,6,30,AL,Baldwin,"Baldwin, AL"
2,1,5,707,0,0,0,AL,Barbour,"Barbour, AL"
3,1,7,780,1,2,8,AL,Bibb,"Bibb, AL"
4,1,9,1979,2,2,8,AL,Blount,"Blount, AL"
...,...,...,...,...,...,...,...,...,...
1116,21,231,885,0,1,0,KY,Wayne,"Wayne, KY"
1117,21,233,564,0,0,0,KY,Webster,"Webster, KY"
1118,21,235,2403,2,6,7,KY,Whitley,"Whitley, KY"
1119,21,237,73,0,0,0,KY,Wolfe,"Wolfe, KY"


In [25]:
# Open 2010 population data
with open('merged_df_2010_pop.pkl', 'rb') as f:
    merged_df_2010_pop = pickle.load(f)

# Create a copy
df_pop_only = merged_df_2010_pop.copy()
df_pop_only.head() #135 counties, #has county_state var

#Merge the two datasets
df_2010 = df_pop_only.merge(crime_2010_filtered, on='county_state', how='inner')
df_2010

,county,schedule1_count,schedule2_count,total_pop,log_slave,ag_output,ag_gdp,gini_x,cash_value_farms,state,...,MEDIAN_AGE_FEM,STNAME_ABBREVIATION,FIPS_ST,FIPS_CTY,GRNDTOT,MURDER,RAPE,MVTHEFT,state_abbrev,county_name_y
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,...,37.9,AL,1,1,2710,5,1,10,AL,Autauga
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,...,42.2,AL,1,3,8503,1,6,30,AL,Baldwin
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,...,41.6,AL,1,5,707,0,0,0,AL,Barbour
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,...,39.5,AL,1,7,780,1,2,8,AL,Bibb
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,...,40.0,AL,1,9,1979,2,2,8,AL,Blount
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2990.0,1823.0,21.0,1844.0,3.044523,85240.0,46.225597,0.30360,474829.0,Georgia,...,40.7,GA,13,299,2278,0,6,36,GA,Ware
131,3130.0,8315.0,170.0,8485.0,5.135798,471886.0,55.614143,0.35042,1818620.0,Georgia,...,35.3,GA,13,313,4879,3,3,24,GA,Whitfield
132,3150.0,1694.0,92.0,1786.0,4.521789,392609.0,219.825867,0.05705,645612.0,Georgia,...,42.4,GA,13,315,89,0,0,0,GA,Wilcox
133,3190.0,5499.0,346.0,5845.0,5.846439,950803.0,162.669464,0.48048,2013324.0,Georgia,...,42.5,GA,13,319,573,0,1,3,GA,Wilkinson


In [26]:
# Calculate crime rates per 100,000 people
df_2010['crime_rate'] = (df_2010['GRNDTOT'] / df_2010['POPESTIMATE']) * 100000
df_2010['murder_rate'] = (df_2010['MURDER'] / df_2010['POPESTIMATE']) * 100000
df_2010['mtv_theft_rate'] = (df_2010['MVTHEFT'] / df_2010['POPESTIMATE']) * 100000

# Display the updated DataFrame with new rate columns
df_2010[['crime_rate', 'murder_rate', 'mtv_theft_rate']].head()

,crime_rate,murder_rate,mtv_theft_rate
0,4966.007586,9.162376,18.324751
1,4665.185307,0.548652,16.459551
2,2574.935353,0.000000,0.000000
3,3403.883919,4.363954,34.911630
4,3452.426642,3.489062,13.956247


In [27]:
df_2010['log_murder_rate'] = np.log(df_2010['murder_rate'] + 1)    # Log transformation for murder rate
df_2010['log_mtv_theft_rate'] = np.log(df_2010['mtv_theft_rate'] + 1)  # Log transformation for motor vehicle theft rate
df_2010['log_crime_rate'] = np.log(df_2010['crime_rate'] + 1)

***2010 Population Density Control***

In [28]:
# Import area data for counties for pop.density control

# Load the data as a DataFrame (use '\t' if it's tab-separated)
df_pop_density_2010 = pd.read_csv("/Users/jakesharadin/2022_Gaz_counties_national.txt", delimiter="\t")  # or ',' if comma-separated

# Display the first few rows
df_pop_density_2010.head()

,USPS,GEOID,ANSICODE,NAME,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001,161526,Autauga County,1539631461,25677536,594.455,9.914,32.532237,-86.646440
1,AL,1003,161527,Baldwin County,4117724893,1132887353,1589.863,437.410,30.659218,-87.746067
2,AL,1005,161528,Barbour County,2292160151,50523213,885.008,19.507,31.870253,-85.405104
3,AL,1007,161529,Bibb County,1612188713,9572302,622.470,3.696,33.015893,-87.127148
4,AL,1009,161530,Blount County,1670259093,14860281,644.891,5.738,33.977358,-86.566440


In [29]:
df_pop_density_2010['county_state'] = df_pop_density_2010['NAME'] + ", " + df_pop_density_2010['USPS']
df_pop_density_2010

,USPS,GEOID,ANSICODE,NAME,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG,county_state
0,AL,1001,161526,Autauga County,1539631461,25677536,594.455,9.914,32.532237,-86.646440,"Autauga County, AL"
1,AL,1003,161527,Baldwin County,4117724893,1132887353,1589.863,437.410,30.659218,-87.746067,"Baldwin County, AL"
2,AL,1005,161528,Barbour County,2292160151,50523213,885.008,19.507,31.870253,-85.405104,"Barbour County, AL"
3,AL,1007,161529,Bibb County,1612188713,9572302,622.470,3.696,33.015893,-87.127148,"Bibb County, AL"
4,AL,1009,161530,Blount County,1670259093,14860281,644.891,5.738,33.977358,-86.566440,"Blount County, AL"
...,...,...,...,...,...,...,...,...,...,...,...
3217,PR,72145,1804553,Vega Baja Municipio,118746310,57826361,45.848,22.327,18.455128,-66.397883,"Vega Baja Municipio, PR"
3218,PR,72147,1804554,Vieques Municipio,131541389,552192821,50.788,213.203,18.125418,-65.432474,"Vieques Municipio, PR"
3219,PR,72149,1804555,Villalba Municipio,92298567,3622640,35.637,1.399,18.130718,-66.472244,"Villalba Municipio, PR"
3220,PR,72151,1804556,Yabucoa Municipio,143005188,72592515,55.215,28.028,18.059858,-65.859871,"Yabucoa Municipio, PR"


In [30]:
# Drop all columns except 'county', 'ALAND_SQMI', and 'AWATER_SQMI', USPS
df_pop_density_filtered = df_pop_density_2010[['county_state', 'ALAND_SQMI', 'AWATER_SQMI']]

# Create the new 'total_sqmi' column as the sum of 'ALAND_SQMI' and 'AWATER_SQMI'
df_pop_density_filtered['total_sqmi'] = df_pop_density_2010['ALAND_SQMI'] + df_pop_density_2010['AWATER_SQMI']

df_pop_density_filtered.head() #3,222 counties

/var/folders/vl/4mchf1rx7l7dljxrjt1ct0_80000gn/T/ipykernel_3012/2230893542.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_density_filtered['total_sqmi'] = df_pop_density_2010['ALAND_SQMI'] + df_pop_density_2010['AWATER_SQMI']


,county_state,ALAND_SQMI,AWATER_SQMI,total_sqmi
0,"Autauga County, AL",594.455,9.914,604.369
1,"Baldwin County, AL",1589.863,437.410,2027.273
2,"Barbour County, AL",885.008,19.507,904.515
3,"Bibb County, AL",622.470,3.696,626.166
4,"Blount County, AL",644.891,5.738,650.629


In [31]:
# Check for duplicates in the same state (none which is good)
duplicate_rows = df_pop_density_filtered[df_pop_density_filtered.duplicated(subset=["county_state"], keep=False)]
print(duplicate_rows)

Empty DataFrame
Columns: [county_state, ALAND_SQMI, AWATER_SQMI, total_sqmi]
Index: []


In [32]:
# Remove the word "County" from each row in the county_state column
df_pop_density_filtered['county_state'] = df_pop_density_filtered['county_state'].str.replace('County', '').str.strip()
df_pop_density_filtered.head()

/var/folders/vl/4mchf1rx7l7dljxrjt1ct0_80000gn/T/ipykernel_3012/3665716905.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_density_filtered['county_state'] = df_pop_density_filtered['county_state'].str.replace('County', '').str.strip()


,county_state,ALAND_SQMI,AWATER_SQMI,total_sqmi
0,"Autauga , AL",594.455,9.914,604.369
1,"Baldwin , AL",1589.863,437.410,2027.273
2,"Barbour , AL",885.008,19.507,904.515
3,"Bibb , AL",622.470,3.696,626.166
4,"Blount , AL",644.891,5.738,650.629


In [33]:
# Remove the space before the comma in county_state column
df_pop_density_filtered['county_state'] = df_pop_density_filtered['county_state'].str.replace(' ,', ',')
df_pop_density_filtered['county_state'].head()

/var/folders/vl/4mchf1rx7l7dljxrjt1ct0_80000gn/T/ipykernel_3012/1455778134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_density_filtered['county_state'] = df_pop_density_filtered['county_state'].str.replace(' ,', ',')


0    Autauga, AL
1    Baldwin, AL
2    Barbour, AL
3       Bibb, AL
4     Blount, AL
Name: county_state, dtype: object

In [34]:
df_2010_control1 = df_2010.merge(df_pop_density_filtered, on="county_state", how="inner")  # or "outer" if you want to keep all rows
df_2010_control1.head() #127 counties

,county,schedule1_count,schedule2_count,total_pop,log_slave,ag_output,ag_gdp,gini_x,cash_value_farms,state,...,county_name_y,crime_rate,murder_rate,mtv_theft_rate,log_murder_rate,log_mtv_theft_rate,log_crime_rate,ALAND_SQMI,AWATER_SQMI,total_sqmi
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,...,Autauga,4966.007586,9.162376,18.324751,2.318692,2.961387,8.510573,594.455,9.914,604.369
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,...,Baldwin,4665.185307,0.548652,16.459551,0.437385,2.859887,8.448097,1589.863,437.410,2027.273
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,...,Barbour,2574.935353,0.000000,0.000000,0.000000,0.000000,7.853968,885.008,19.507,904.515
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,...,Bibb,3403.883919,4.363954,34.911630,1.679701,3.581061,8.132966,622.470,3.696,626.166
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,...,Blount,3452.426642,3.489062,13.956247,1.501644,2.705129,8.147122,644.891,5.738,650.629


In [35]:
# Create the pop_density column
df_2010_control1['pop_density_2010'] = df_2010_control1['POPESTIMATE'] / df_2010_control1['total_sqmi']

***Load 'pop_density_1860' 'males_15_29_total_1860'***

In [36]:
# Load 1860 county area data
# Use 2010 county area size FOR NOW

# Load 1860 males aged 15-29 data
males15_29_1860 = pd.read_stata("/Users/jakesharadin/Desktop/berkeley/2024_3_fall/econ_191/research/data/controls/age/males_15_29_1860.dta")
males15_29_1860.head() #206 counties

,county,total_males_15_29_by_county
0,10,50227.0
1,30,42867.0
2,50,55791.0
3,70,21539.0
4,90,56997.0


***Merge final control1 df***

In [37]:
df_2010_controls2_final = df_2010_control1.merge(males15_29_1860, on="county", how="inner")  # or "outer" if you want to keep all rows
df_2010_controls2_final.rename(columns={'total_males_15_29_by_county': 'total_males_15_29_1860'}, inplace=True)
df_2010_controls2_final.rename(columns={'total_pop': 'total_pop_1860'}, inplace=True)
df_2010_controls2_final.rename(columns={'POPESTIMATE': 'total_pop_2010'}, inplace=True)
df_2010_controls2_final.head() #129 observations (good)
#df_2010_control1_final.shape

,county,schedule1_count,schedule2_count,total_pop_1860,log_slave,ag_output,ag_gdp,gini_x,cash_value_farms,state,...,murder_rate,mtv_theft_rate,log_murder_rate,log_mtv_theft_rate,log_crime_rate,ALAND_SQMI,AWATER_SQMI,total_sqmi,pop_density_2010,total_males_15_29_1860
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,...,9.162376,18.324751,2.318692,2.961387,8.510573,594.455,9.914,604.369,90.294175,50227.0
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,...,0.548652,16.459551,0.437385,2.859887,8.448097,1589.863,437.410,2027.273,89.906490,42867.0
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,...,0.000000,0.000000,0.000000,0.000000,7.853968,885.008,19.507,904.515,30.355494,55791.0
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,...,4.363954,34.911630,1.679701,3.581061,8.132966,622.470,3.696,626.166,36.595727,21539.0
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,...,3.489062,13.956247,1.501644,2.705129,8.147122,644.891,5.738,650.629,88.102436,56997.0


In [38]:
for col in df_2010_controls2_final.columns:
    print(col)

county
schedule1_count
schedule2_count
total_pop_1860
log_slave
ag_output
ag_gdp
gini_x
cash_value_farms
state
county_name_x
log_pop
gini_stata
state_abb
county_state
county_y
STNAME
CTYNAME
YEAR
total_pop_2010
POPEST_MALE
POPEST_FEM
UNDER5_TOT
UNDER5_MALE
UNDER5_FEM
AGE513_TOT
AGE513_MALE
AGE513_FEM
AGE1417_TOT
AGE1417_MALE
AGE1417_FEM
AGE1824_TOT
AGE1824_MALE
AGE1824_FEM
AGE16PLUS_TOT
AGE16PLUS_MALE
AGE16PLUS_FEM
AGE18PLUS_TOT
AGE18PLUS_MALE
AGE18PLUS_FEM
AGE1544_TOT
AGE1544_MALE
AGE1544_FEM
AGE2544_TOT
AGE2544_MALE
AGE2544_FEM
AGE4564_TOT
AGE4564_MALE
AGE4564_FEM
AGE65PLUS_TOT
AGE65PLUS_MALE
AGE65PLUS_FEM
AGE04_TOT
AGE04_MALE
AGE04_FEM
AGE59_TOT
AGE59_MALE
AGE59_FEM
AGE1014_TOT
AGE1014_MALE
AGE1014_FEM
AGE1519_TOT
AGE1519_MALE
AGE1519_FEM
AGE2024_TOT
AGE2024_MALE
AGE2024_FEM
AGE2529_TOT
AGE2529_MALE
AGE2529_FEM
AGE3034_TOT
AGE3034_MALE
AGE3034_FEM
AGE3539_TOT
AGE3539_MALE
AGE3539_FEM
AGE4044_TOT
AGE4044_MALE
AGE4044_FEM
AGE4549_TOT
AGE4549_MALE
AGE4549_FEM
AGE5054_TOT
AGE5054_MALE
A

In [39]:
df_2010_controls2_final['pop_density_1860'] = df_2010_controls2_final['total_pop_1860'] / df_2010_controls2_final['total_sqmi']
df_2010_controls2_final.head()

,county,schedule1_count,schedule2_count,total_pop_1860,log_slave,ag_output,ag_gdp,gini_x,cash_value_farms,state,...,mtv_theft_rate,log_murder_rate,log_mtv_theft_rate,log_crime_rate,ALAND_SQMI,AWATER_SQMI,total_sqmi,pop_density_2010,total_males_15_29_1860,pop_density_1860
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,...,18.324751,2.318692,2.961387,8.510573,594.455,9.914,604.369,90.294175,50227.0,601.796254
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,...,16.459551,0.437385,2.859887,8.448097,1589.863,437.410,2027.273,89.906490,42867.0,144.973075
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,...,0.000000,0.000000,0.000000,7.853968,885.008,19.507,904.515,30.355494,55791.0,438.918094
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,...,34.911630,1.679701,3.581061,8.132966,622.470,3.696,626.166,36.595727,21539.0,221.572235
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,...,13.956247,1.501644,2.705129,8.147122,644.891,5.738,650.629,88.102436,56997.0,578.180499


In [40]:
df_2010_controls2_final['total_males_15_29_2010'] = (
    df_2010_controls2_final['AGE1519_MALE'] + 
    df_2010_controls2_final['AGE2024_MALE'] + 
    df_2010_controls2_final['AGE2529_MALE']
)

In [41]:
df_2010_controls2_final.rename(columns={'ag_gdp': 'ag_gdp_1860'}, inplace=True)

In [42]:
# Rename the column log_pop to log_pop_1860
df_2010_controls2_final.rename(columns={'log_pop': 'log_pop_1860'}, inplace=True)

In [43]:
df_2010_controls2_final.head()

,county,schedule1_count,schedule2_count,total_pop_1860,log_slave,ag_output,ag_gdp_1860,gini_x,cash_value_farms,state,...,log_murder_rate,log_mtv_theft_rate,log_crime_rate,ALAND_SQMI,AWATER_SQMI,total_sqmi,pop_density_2010,total_males_15_29_1860,pop_density_1860,total_males_15_29_2010
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,...,2.318692,2.961387,8.510573,594.455,9.914,604.369,90.294175,50227.0,601.796254,5295
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,...,0.437385,2.859887,8.448097,1589.863,437.410,2027.273,89.906490,42867.0,144.973075,15906
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,...,0.000000,0.000000,7.853968,885.008,19.507,904.515,30.355494,55791.0,438.918094,3179
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,...,1.679701,3.581061,8.132966,622.470,3.696,626.166,36.595727,21539.0,221.572235,2604
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,...,1.501644,2.705129,8.147122,644.891,5.738,650.629,88.102436,56997.0,578.180499,5451


In [44]:
# Real GDP recent data

In [51]:
real_gdp = pd.read_csv("/Users/jakesharadin/Desktop/CAGDP1__ALL_AREAS_2001_2023.csv", encoding='latin1')  # Or 'ISO-8859-1'
real_gdp = real_gdp.drop(columns=['GeoFIPS', 'Region', 'LineCode', 'TableName'])

real_gdp.head()

,GeoName,IndustryClassification,Description,Unit,2001,2002,2003,2004,2005,2006,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,United States,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,14230726000,14472712000,14877312000,15449757000,15987957000,16433148000,...,18261714000,18799622000,19141672000,19612102000,20193896000,20715671000,20267585000,21494798000,22034828000,22671096000
1,United States,...,Chain-type quantity indexes for real GDP,Quantity index,72.561,73.795,75.858,78.777,81.521,83.791,...,93.115,95.857,97.601,100.000,102.967,105.627,103.342,109.600,112.353,115.597
2,United States,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,10581929000,10929108000,11456450000,12217196000,13039197000,13815583000,...,17608138000,18295019000,18804913000,19612102000,20656516000,21539982000,21354105000,23681171000,26006893000,27720709000
3,Alabama,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,168448296,172430379,178039962,189913745,197246483,199923857,...,206070026,208950294,212862789,216615470,220808767,225272775,222288753,233726620,238556458,245354674
4,Alabama,...,Chain-type quantity indexes for real GDP,Quantity index,77.764,79.602,82.192,87.673,91.058,92.294,...,95.132,96.461,98.268,100.000,101.936,103.997,102.619,107.899,110.129,113.267


In [52]:
# Filter the DataFrame to keep only rows where 'Description' equals the specified value
real_gdp = real_gdp[real_gdp['Description'] == 'Current-dollar GDP (thousands of current dollars) ']

# Display the resulting DataFrame
real_gdp

,GeoName,IndustryClassification,Description,Unit,2001,2002,2003,2004,2005,2006,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
2,United States,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,10581929000,10929108000,11456450000,12217196000,13039197000,13815583000,...,17608138000,18295019000,18804913000,19612102000,20656516000,21539982000,21354105000,23681171000,26006893000,27720709000
5,Alabama,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,123534517,128380896,135212626,148379077,158677285,166048211,...,197064403,203113340,208824280,216615470,226263784,234798213,235325321,260018429,284949136,304935567
8,"Autauga, AL",...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,760973,800045,834208,984461,1015320,1124089,...,1569120,1729098,1806246,1762558,1826642,1814228,1841867,1954645,2318914,2452642
11,"Baldwin, AL",...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,3128482,3369358,3642003,4117129,4717641,5031537,...,6034727,6492574,6983037,7382558,7935575,8558537,8770138,9847424,11097699,12071468
14,"Barbour, AL",...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,642028,637765,664978,741585,748820,744004,...,779299,765000,757473,761761,789103,793769,787697,868828,931174,926685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9521,Plains,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,673288427,699597124,738452832,788026218,831003115,867930312,...,1159084109,1196563655,1217270085,1250759343,1301099969,1340848043,1336823238,1476248606,1620951934,1726903182
9524,Southeast,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,2276728106,2373091556,2506652905,2698495596,2913203602,3083770790,...,3726579792,3901056533,4037867547,4210089181,4411751306,4611953719,4609791490,5132905366,5685333127,6122626696
9527,Southwest,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,1106271358,1134836906,1204221734,1313307930,1431836022,1570658164,...,2168451055,2163072239,2169483346,2286160711,2464729990,2550062656,2484272963,2856515882,3295092219,3498332461
9530,Rocky Mountain,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,338172137,347931550,362401145,386799833,423474812,456519984,...,602131039,621890048,641047020,681310123,730567674,776330214,780157535,880955513,986528188,1057159025


In [53]:
real_gdp = real_gdp.drop(columns=['IndustryClassification', 'Description', 'Unit'])
real_gdp = real_gdp.rename(columns={str(year): f'gdp_{year}' for year in range(2001, 2024)})
real_gdp.head()


,GeoName,gdp_2001,gdp_2002,gdp_2003,gdp_2004,gdp_2005,gdp_2006,gdp_2007,gdp_2008,gdp_2009,...,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,gdp_2019,gdp_2020,gdp_2021,gdp_2022,gdp_2023
2,United States,10581929000,10929108000,11456450000,12217196000,13039197000,13815583000,14474228000,14769862000,14478067000,...,17608138000,18295019000,18804913000,19612102000,20656516000,21539982000,21354105000,23681171000,26006893000,27720709000
5,Alabama,123534517,128380896,135212626,148379077,158677285,166048211,172246995,174665564,170876463,...,197064403,203113340,208824280,216615470,226263784,234798213,235325321,260018429,284949136,304935567
8,"Autauga, AL",760973,800045,834208,984461,1015320,1124089,1182234,1097703,1175769,...,1569120,1729098,1806246,1762558,1826642,1814228,1841867,1954645,2318914,2452642
11,"Baldwin, AL",3128482,3369358,3642003,4117129,4717641,5031537,5330102,5238424,5103539,...,6034727,6492574,6983037,7382558,7935575,8558537,8770138,9847424,11097699,12071468
14,"Barbour, AL",642028,637765,664978,741585,748820,744004,739799,713602,721550,...,779299,765000,757473,761761,789103,793769,787697,868828,931174,926685


In [54]:
real_gdp = real_gdp.rename(columns={'GeoName': 'county_state'})

# Perform an inner merge on 'county' and 'state'
df_2010_controls2_final = pd.merge(
    df_2010_controls2_final,  # Second dataset
    real_gdp,                # First dataset
    on=['county_state'],  # Columns to match on
    how='inner'              # Perform an inner merge
)

# Display the resulting DataFrame
df_2010_controls2_final.head() #122 observations

,county,schedule1_count,schedule2_count,total_pop_1860,log_slave,ag_output,ag_gdp_1860,gini_x,cash_value_farms,state,...,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,gdp_2019,gdp_2020,gdp_2021,gdp_2022,gdp_2023
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,...,1569120,1729098,1806246,1762558,1826642,1814228,1841867,1954645,2318914,2452642
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,...,6034727,6492574,6983037,7382558,7935575,8558537,8770138,9847424,11097699,12071468
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,...,779299,765000,757473,761761,789103,793769,787697,868828,931174,926685
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,...,381354,377535,392287,406741,408791,465977,502089,530401,571231,610429
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,...,928552,987510,925988,987967,1064218,1055980,973485,1162103,1266269,1261107


In [55]:
# Identify columns that start with 'gdp_'
gdp_columns = [col for col in df_2010_controls2_final.columns if col.startswith('gdp_')]

# Convert all GDP columns to numeric
df_2010_controls2_final[gdp_columns] = df_2010_controls2_final[gdp_columns].apply(pd.to_numeric, errors='coerce')

# Check for NaN values introduced during the conversion
print(df_2010_controls2_final[gdp_columns].isna().sum())


gdp_2001    0
gdp_2002    0
gdp_2003    0
gdp_2004    0
gdp_2005    0
gdp_2006    0
gdp_2007    0
gdp_2008    0
gdp_2009    0
gdp_2010    0
gdp_2011    0
gdp_2012    0
gdp_2013    0
gdp_2014    0
gdp_2015    0
gdp_2016    0
gdp_2017    0
gdp_2018    0
gdp_2019    0
gdp_2020    0
gdp_2021    0
gdp_2022    0
gdp_2023    0
dtype: int64


In [56]:
# Remove California and Illinois from the dataset since they're not slave states 
df_2010_controls2_final = df_2010_controls2_final[~df_2010_controls2_final['state'].isin(['California', 'Illinois'])]
df_2010_controls2_final #126 observations

,county,schedule1_count,schedule2_count,total_pop_1860,log_slave,ag_output,ag_gdp_1860,gini_x,cash_value_farms,state,...,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,gdp_2019,gdp_2020,gdp_2021,gdp_2022,gdp_2023
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,...,1569120,1729098,1806246,1762558,1826642,1814228,1841867,1954645,2318914,2452642
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,...,6034727,6492574,6983037,7382558,7935575,8558537,8770138,9847424,11097699,12071468
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,...,779299,765000,757473,761761,789103,793769,787697,868828,931174,926685
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,...,381354,377535,392287,406741,408791,465977,502089,530401,571231,610429
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,...,928552,987510,925988,987967,1064218,1055980,973485,1162103,1266269,1261107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2990.0,1823.0,21.0,1844.0,3.044523,85240.0,46.225597,0.30360,474829.0,Georgia,...,1235393,1329663,1389632,1456309,1501530,1493006,1449183,1561181,1736077,1799317
131,3130.0,8315.0,170.0,8485.0,5.135798,471886.0,55.614143,0.35042,1818620.0,Georgia,...,5247700,5442186,5614443,5603597,5742823,5867997,5765571,6329940,7114984,7370579
132,3150.0,1694.0,92.0,1786.0,4.521789,392609.0,219.825867,0.05705,645612.0,Georgia,...,119518,145592,142459,159134,153045,161843,147671,165637,196414,191013
133,3190.0,5499.0,346.0,5845.0,5.846439,950803.0,162.669464,0.48048,2013324.0,Georgia,...,462955,420221,429228,507488,525398,565169,582965,694210,754792,729725


In [57]:
# Add crime data from 1990
ninety_crime = pd.read_stata("/Users/jakesharadin/Desktop/berkeley/2024_3_fall/econ_191/research/data/crime/1990/ICPSR_1990/DS0001/1990_crime.dta")

ninety_crime = ninety_crime.rename(columns={
    'V5': 'state_code',
    'V6': 'county_code',
    'V10': 'grand_total_crimes_1990',
    'V14': 'murders_1990',
    'V20': 'vehicle_thefts_1990'  
})

ninety_crime.head()

,state_code,county_code,grand_total_crimes_1990,murders_1990,vehicle_thefts_1990
0,1,1,1396,0,5
1,1,3,4186,2,22
2,1,5,899,3,7
3,1,7,0,0,0
4,1,9,1168,2,11


In [58]:
# Filter rows in ninety_crime where state_code exists in county_fips_names
valid_states = set(county_fips_names.keys())

# Remove rows with invalid state codes
ninety_crime = ninety_crime[ninety_crime['state_code'].isin(valid_states)]

# Function to check if a county exists in the county_fips_names dictionary for a given state
def is_valid_county(row):
    state_code = row['state_code']
    county_code = row['county_code']
    return county_code in county_fips_names.get(state_code, {})

# Remove rows with invalid county codes
ninety_crime = ninety_crime[ninety_crime.apply(is_valid_county, axis=1)]

# Map state abbreviations and county names to create "county_state"
ninety_crime['state_abb'] = ninety_crime['state_code'].map(lambda x: state_fips_abbrev.get(x, ""))
ninety_crime['county_name'] = ninety_crime.apply(
    lambda row: county_fips_names.get(row['state_code'], {}).get(row['county_code'], ""), axis=1
)

# Create the "county_state" column
ninety_crime['county_state'] = ninety_crime['county_name'] + ', ' + ninety_crime['state_abb']

# Remove rows where "county_state" is not valid (empty values)
ninety_crime = ninety_crime[ninety_crime['county_state'].str.strip() != ',']
ninety_crime.head()

,state_code,county_code,grand_total_crimes_1990,murders_1990,vehicle_thefts_1990,state_abb,county_name,county_state
0,1,1,1396,0,5,AL,Autauga,"Autauga, AL"
1,1,3,4186,2,22,AL,Baldwin,"Baldwin, AL"
2,1,5,899,3,7,AL,Barbour,"Barbour, AL"
3,1,7,0,0,0,AL,Bibb,"Bibb, AL"
4,1,9,1168,2,11,AL,Blount,"Blount, AL"


In [59]:
ninety_crime = ninety_crime.drop(columns=['state_code', 'county_code','state_abb', 'county_name'])


In [60]:
ninety_crime.head()

,grand_total_crimes_1990,murders_1990,vehicle_thefts_1990,county_state
0,1396,0,5,"Autauga, AL"
1,4186,2,22,"Baldwin, AL"
2,899,3,7,"Barbour, AL"
3,0,0,0,"Bibb, AL"
4,1168,2,11,"Blount, AL"


In [61]:
# Add crime data from 2000
two_thousand_crime = pd.read_stata("/Users/jakesharadin/Desktop/berkeley/2024_3_fall/econ_191/research/data/crime/2000/ICPSR_2000/DS0001/2000_crime.dta")

two_thousand_crime = two_thousand_crime.rename(columns={
    'FIPS_ST': 'state_code',
    'FIPS_CTY': 'county_code',
    'GRNDTOT': 'grand_total_crimes_2000',
    'MURDER': 'murders_2000',
    'MVTHEFT': 'vehicle_thefts_2000'  
})

two_thousand_crime.head()

,state_code,county_code,grand_total_crimes_2000,murders_2000,vehicle_thefts_2000
0,1,1,2433,2,24
1,1,3,8881,9,35
2,1,5,1652,2,7
3,1,7,830,1,4
4,1,9,1447,0,7


In [62]:
# Filter rows in ninety_crime where state_code exists in county_fips_names
valid_states = set(county_fips_names.keys())

# Remove rows with invalid state codes
two_thousand_crime = two_thousand_crime[two_thousand_crime['state_code'].isin(valid_states)]

# Function to check if a county exists in the county_fips_names dictionary for a given state
def is_valid_county(row):
    state_code = row['state_code']
    county_code = row['county_code']
    return county_code in county_fips_names.get(state_code, {})

# Remove rows with invalid county codes
two_thousand_crime = two_thousand_crime[two_thousand_crime.apply(is_valid_county, axis=1)]

# Map state abbreviations and county names to create "county_state"
two_thousand_crime['state_abb'] = two_thousand_crime['state_code'].map(lambda x: state_fips_abbrev.get(x, ""))
two_thousand_crime['county_name'] = two_thousand_crime.apply(
    lambda row: county_fips_names.get(row['state_code'], {}).get(row['county_code'], ""), axis=1
)

# Create the "county_state" column
two_thousand_crime['county_state'] = two_thousand_crime['county_name'] + ', ' + two_thousand_crime['state_abb']

# Remove rows where "county_state" is not valid (empty values)
two_thousand_crime = two_thousand_crime[two_thousand_crime['county_state'].str.strip() != ',']
two_thousand_crime.head()

,state_code,county_code,grand_total_crimes_2000,murders_2000,vehicle_thefts_2000,state_abb,county_name,county_state
0,1,1,2433,2,24,AL,Autauga,"Autauga, AL"
1,1,3,8881,9,35,AL,Baldwin,"Baldwin, AL"
2,1,5,1652,2,7,AL,Barbour,"Barbour, AL"
3,1,7,830,1,4,AL,Bibb,"Bibb, AL"
4,1,9,1447,0,7,AL,Blount,"Blount, AL"


In [63]:
two_thousand_crime = two_thousand_crime.drop(columns=['state_code', 'county_code','state_abb', 'county_name'])

In [64]:
two_thousand_crime.head()

,grand_total_crimes_2000,murders_2000,vehicle_thefts_2000,county_state
0,2433,2,24,"Autauga, AL"
1,8881,9,35,"Baldwin, AL"
2,1652,2,7,"Barbour, AL"
3,830,1,4,"Bibb, AL"
4,1447,0,7,"Blount, AL"


In [65]:
ninety_crime.head()

,grand_total_crimes_1990,murders_1990,vehicle_thefts_1990,county_state
0,1396,0,5,"Autauga, AL"
1,4186,2,22,"Baldwin, AL"
2,899,3,7,"Barbour, AL"
3,0,0,0,"Bibb, AL"
4,1168,2,11,"Blount, AL"


In [66]:
# Strip and standardize column names
two_thousand_crime.columns = two_thousand_crime.columns.str.strip()
ninety_crime.columns = ninety_crime.columns.str.strip()

# Merge on county_state
merged_crime = pd.merge(two_thousand_crime, ninety_crime, on='county_state', how='inner')

# Display the result
merged_crime.head()

,grand_total_crimes_2000,murders_2000,vehicle_thefts_2000,county_state,grand_total_crimes_1990,murders_1990,vehicle_thefts_1990
0,2433,2,24,"Autauga, AL",1396,0,5
1,8881,9,35,"Baldwin, AL",4186,2,22
2,1652,2,7,"Barbour, AL",899,3,7
3,830,1,4,"Bibb, AL",0,0,0
4,1447,0,7,"Blount, AL",1168,2,11


In [67]:
df_2010_controls2_final.head()

,county,schedule1_count,schedule2_count,total_pop_1860,log_slave,ag_output,ag_gdp_1860,gini_x,cash_value_farms,state,...,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,gdp_2019,gdp_2020,gdp_2021,gdp_2022,gdp_2023
0,10.0,360232.0,3475.0,363707.0,8.153350,32662909.0,89.805557,0.54031,129368094.0,Alabama,...,1569120,1729098,1806246,1762558,1826642,1814228,1841867,1954645,2318914,2452642
1,30.0,292363.0,1537.0,293900.0,7.337588,27996221.0,95.257645,0.58392,152052217.0,Alabama,...,6034727,6492574,6983037,7382558,7935575,8558537,8770138,9847424,11097699,12071468
2,50.0,395136.0,1872.0,397008.0,7.534763,31948684.0,80.473656,0.52155,142857209.0,Alabama,...,779299,765000,757473,761761,789103,793769,787697,868828,931174,926685
3,70.0,136102.0,2639.0,138741.0,7.878155,24998235.0,180.179153,0.43734,99711086.0,Alabama,...,381354,377535,392287,406741,408791,465977,502089,530401,571231,610429
4,90.0,375103.0,1078.0,376181.0,6.982863,21747586.0,57.811497,0.59723,88331569.0,Alabama,...,928552,987510,925988,987967,1064218,1055980,973485,1162103,1266269,1261107


In [70]:
# Merge with df_2010_controls2_final

# Merge on county_state
df_2010_controls2_final_copy = pd.merge(df_2010_controls2_final, merged_crime, on='county_state', how='inner')

# Display the result
df_2010_controls2_final_copy.head()
df_2010_controls2_final = df_2010_controls2_final_copy

In [76]:
# List of columns to keep
columns_to_keep = ['county_state', 'log_slave', 'gini_stata', 'log_murder_rate', 
                   'log_mtv_theft_rate', 'log_crime_rate', 'pop_density_1860', 
                   'pop_density_2010', 'total_males_15_29_1860', 
                   'total_males_15_29_2010', 'ag_gdp_1860', 'gdp_2010']

# Drop all columns except those in columns_to_keep
summary_stats = df_2010_controls2_final[columns_to_keep]
summary_stats.describe()

,log_slave,gini_stata,log_murder_rate,log_mtv_theft_rate,log_crime_rate,pop_density_1860,pop_density_2010,total_males_15_29_1860,total_males_15_29_2010,ag_gdp_1860,gdp_2010
count,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,1.260000e+02
mean,6.780058,0.468754,1.095024,2.731735,8.159430,122.140227,122.191932,11384.650391,6883.547619,177.230118,2.427855e+06
std,1.194405,0.144449,0.987435,1.047625,0.580249,128.366296,213.124132,12329.900391,11820.113720,77.295433,5.285098e+06
min,2.564949,0.019370,0.000000,0.000000,5.668589,2.030866,8.789487,224.000000,158.000000,39.990108,2.454400e+04
25%,6.057328,0.417863,0.000000,2.467355,7.912224,49.606612,29.640100,2013.250000,1474.750000,110.870916,3.630868e+05
50%,7.104141,0.502470,1.307484,2.903547,8.212991,90.536214,48.382700,8777.000000,2617.000000,167.306099,7.407100e+05
75%,7.660880,0.567577,1.814591,3.363807,8.513644,139.865788,107.533061,13563.000000,6214.750000,225.000904,1.883910e+06
max,8.501674,0.715920,3.579041,4.716961,9.549460,752.799071,1843.725818,56997.000000,84257.000000,454.349365,3.746958e+07


In [71]:
# Save df_2010 to a pickle file
with open('df_2010_controls2_final.pkl', 'wb') as f:
    pickle.dump(df_2010_controls2_final, f)